In [2]:
import os
os.chdir('..')

In [3]:
from src.multimodal.model import MultimodalModel
from src.utils import get_lookup, load_pkl, get_labels
from src.text.model.word_embedding import WordEmbedding
from src.text.model.custom_dataset import CustomDataset
from src.mol.preprocess import mapped_smiles_reader, candidate_smiles
from src.mol.mol_dataset import MolDataset

## Text

In [4]:
lookup_word = get_lookup('cache/fasttext/nguyennb/all_words.txt')
lookup_tag = get_lookup('cache/fasttext/nguyennb/all_pos.txt')
lookup_dep = get_lookup('cache/fasttext/nguyennb/all_dep.txt')
lookup_direction = get_lookup('cache/fasttext/nguyennb/all_direction.txt')

all_candidates_train = load_pkl('cache/pkl/v1/candidates.train.pkl')
all_candidates_test = load_pkl('cache/pkl/v1/candidates.test.pkl')
sdp_train = load_pkl('cache/pkl/v1/sdp.train.pkl')
sdp_test = load_pkl('cache/pkl/v1/sdp.test.pkl')
sdp_train_mapped = load_pkl('cache/pkl/v1/train.mapped.sdp.pkl')
sdp_test_mapped = load_pkl('cache/pkl/v1/test.mapped.sdp.pkl')
we = WordEmbedding(fasttext_path='cache/fasttext/nguyennb/fastText_ddi.npz',
                   vocab_path='cache/fasttext/nguyennb/all_words.txt')

y_train = get_labels(all_candidates_train)
y_test = get_labels(all_candidates_test)

In [5]:
data_train = CustomDataset(sdp_train_mapped, y_train)
data_train.fix_exception()
data_train.batch_padding(batch_size=1)
data_train.squeeze()
data_test = CustomDataset(sdp_test_mapped, y_test)
data_test.fix_exception()
data_test.batch_padding(batch_size=1)
data_test.squeeze()

In [6]:
from torch.utils.data import DataLoader

dataloader_train = DataLoader(data_train, batch_size=1)
dataloader_test = DataLoader(data_test, batch_size=1)

## Mol

In [7]:
all_candidates_train = load_pkl('cache/pkl/v1/candidates.train.pkl')
all_candidates_test = load_pkl('cache/pkl/v1/candidates.test.pkl')
mapped_smiles = mapped_smiles_reader('cache/mapped_drugs/all_mapped.txt')
x_train, y_train = candidate_smiles(all_candidates_train, mapped_smiles)
x_test, y_test = candidate_smiles(all_candidates_test, mapped_smiles)
dataset_train = MolDataset(x_train, y_train)
dataset_test = MolDataset(x_test, y_test)

Converting SMILES to PyG: 100%|██████████| 3374/3374 [00:08<00:00, 406.07it/s]


## Training